# 在函数上添加包装器

In [1]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

# 创建装饰器时保留函数元信息

`@wraps` 有一个重要特征是它能让你通过属性 `__wrapped__` 直接访问被包装函数。

In [3]:
@timethis
def countdown(n):
     '''
     Counts down
     '''
     while n > 0:
        n -= 1

In [4]:
from inspect import signature
print(signature(countdown))

(n)


# 解除一个装饰器

In [6]:
@timethis
def add(x, y):
    return x + y

orig_add = add.__wrapped__
orig_add(3, 4)

7

In [7]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def add(x, y):
    return x + y

In [8]:
add(2, 3)

Decorator 1
Decorator 2


5

In [9]:
add.__wrapped__(2,3)

Decorator 2


5

In [10]:
add.__wrapped__.__wrapped__(2,3)

5

# 定义一个带参数的装饰器

In [11]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

# 可自定义属性的装饰器

In [16]:
from functools import wraps, partial
import logging
# Utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        # Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper

    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [13]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [14]:
add(2,3)

DEBUG:__main__:add


5

In [17]:
add.set_message('Add called')
add(2,3)

DEBUG:__main__:Add called


5

In [18]:
add.set_level(logging.WARNING)
add(2,3)

5

# 带可选参数的装饰器

In [19]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__

    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)

    return wrapper

# Example use
@logged
def add(x, y):
    return x + y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')

# 利用装饰器强制函数上的类型检查

In [20]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func

        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                            )
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [21]:
from inspect import signature
def spam(x,y,z=42):pass
sig = signature(spam)
sig

<Signature (x, y, z=42)>

In [22]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [24]:
bound_types = sig.bind_partial(int,z=int)
bound_types

<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

In [25]:
bound_types.arguments

OrderedDict([('x', int), ('z', int)])

In [27]:
bound_values  = sig.bind(1,2,3)
bound_values

<BoundArguments (x=1, y=2, z=3)>

In [28]:
@typeassert(int, int)
def add(x, y):
    return x + y
add(1,'s')

TypeError: Argument y must be <class 'int'>

# 将装饰器定义为类的一部分

In [29]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper

    # Decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper

In [30]:
# As an instance method
a = A()
@a.decorator1
def spam():
    pass
# As a class method
@A.decorator2
def grok():
    pass

# 将装饰器定义为类

将装饰器定义成一个实例，你需要确保它实现了 `__call__()` 和 `__get__()` 方法

In [32]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        print('__init__')
        self.ncalls = 0

    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        print('__call__')
        return self.__wrapped__(*args, **kwargs)

    def __get__(self, instance, cls):
        print('__get__')
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)

In [33]:
@Profiled
def add(x, y):
    return x + y

__init__


In [34]:
add(1,2)

__call__


3

In [35]:
class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)

__init__


In [36]:
s = Spam()

In [37]:
s.bar(1)

__get__
__call__
<__main__.Spam object at 0x7fe1340bea90> 1


# 为类和静态方法提供装饰器

给类或静态方法提供装饰器是很简单的，不过要确保装饰器在 `@classmethod` 或 `@staticmethod` 之前  
在于 `@classmethod` 和 `@staticmethod` 实际上并不会创建可直接调用的对象， 而是创建特殊的描述器对象

In [39]:
import time
from functools import wraps

# A simple decorator
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(end-start)
        return r
    return wrapper

# Class illustrating application of the decorator to different kinds of methods
class Spam:
    @timethis
    def instance_method(self, n):
        print(self, n)
        while n > 0:
            n -= 1

    @classmethod
    @timethis
    def class_method(cls, n):
        print(cls, n)
        while n > 0:
            n -= 1

    @staticmethod
    @timethis
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1

# 装饰器为被包装函数增加参数

## 增加参数

In [40]:
from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)

    return wrapper

## 修正函数签名

In [41]:
from functools import wraps
import inspect

def optional_debug(func):
    if 'debug' in inspect.getargspec(func).args:
        raise TypeError('debug argument already defined')

    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)

    sig = inspect.signature(func)
    parms = list(sig.parameters.values())
    parms.append(inspect.Parameter('debug',
                inspect.Parameter.KEYWORD_ONLY,
                default=False))
    wrapper.__signature__ = sig.replace(parameters=parms)
    return wrapper

# 使用装饰器扩充类的功能

In [44]:
def log_getattribute(cls):
    # Get the original implementation
    orig_getattribute = cls.__getattribute__

    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)

    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls

# Example use
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass

In [45]:
a = A(42)

In [46]:
a.x

getting: x


42

In [47]:
a.spam()

getting: spam


# 使用元类控制实例的创建

## 不能创建实例的类

In [48]:
class NoInstances(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")

# Example
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

In [49]:
s = Spam()

TypeError: Can't instantiate directly

## 单例模式

In [50]:
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs)
            return self.__instance
        else:
            return self.__instance

# Example
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')

## 缓存实例

In [51]:
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()

    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj

# Example
class Spam(metaclass=Cached):
    def __init__(self, name):
        print('Creating Spam({!r})'.format(name))
        self.name = name

# 捕获类的属性定义顺序

` __new__()` 方法中对于元类中被修改字典的处理。 尽管类使用了另外一个字典来定义，在构造最终的 `class`对象的时候， 我们仍然需要将这个字典转换为一个正确的 `dict` 实例。 通过语句 `d = dict(clsdict)` 来完成这个效果  
`__prepare__()` 方法。 这个方法会在开始定义类和它的父类的时候被执行。它必须返回一个映射对象以便在类定义体中被使用到。

## 定义顺序

In [54]:
from collections import OrderedDict

# A set of descriptors for various types
class Typed:
    _expected_type = type(None)
    def __init__(self, name=None):
        self._name = name

    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' + str(self._expected_type))
        instance.__dict__[self._name] = value

class Integer(Typed):
    _expected_type = int

class Float(Typed):
    _expected_type = float

class String(Typed):
    _expected_type = str

# Metaclass that uses an OrderedDict for class body
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        order = []
        for name, value in clsdict.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)

    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()

In [55]:
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)

# Example use
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [56]:
s = Stock('GOOG',100,490.1)
s.name

'GOOG'

In [58]:
s.as_csv()

'GOOG,100,490.1'

## 防止重复定义

In [59]:
from collections import OrderedDict

class NoDupOrderedDict(OrderedDict):
    def __init__(self, clsname):
        self.clsname = clsname
        super().__init__()
    def __setitem__(self, name, value):
        if name in self:
            raise TypeError('{} already defined in {}'.format(name, self.clsname))
        super().__setitem__(name, value)

class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        d['_order'] = [name for name in clsdict if name[0] != '_']
        return type.__new__(cls, clsname, bases, d)

    @classmethod
    def __prepare__(cls, clsname, bases):
        return NoDupOrderedDict(clsname)

In [60]:
class A(metaclass=OrderedMeta):
    def spam(self):
        pass
    def spam(self):
        pass

TypeError: spam already defined in A

# 定义有可选参数的元类

为了使元类支持这些关键字参数，你必须确保在 `__prepare__()` , `__new__()` 和 `__init__()` 方法中 都使用强制关键字参数。

In [63]:
class MyMeta(type):
    # Optional
    @classmethod
    def __prepare__(cls, name, bases, *, debug=False, synchronize=False):
        # Custom processing
        pass
        return super().__prepare__(name, bases)

    # Required
    def __new__(cls, name, bases, ns, *, debug=False, synchronize=False):
        # Custom processing
        pass
        return super().__new__(cls, name, bases, ns)

    # Required
    def __init__(self, name, bases, ns, *, debug=False, synchronize=False):
        # Custom processing
        pass
        super().__init__(name, bases, ns)

`__prepare__()` 方法在所有类定义开始执行前首先被调用，用来创建类命名空间。通常来讲，这个方法只是简单的返回一个字典或其他映射对象。  
 `__new__()` 方法被用来实例化最终的类对象。它在类的主体被执行完后开始执行。   
 `__init__()` 方法最后被调用，用来执行其他的一些初始化工作。

将这些属性定义为参数的好处在于它们不会污染类的名称空间， 这些属性仅仅只从属于类的创建阶段，而不是类中的语句执行阶段。  
另外，它们在 `__prepare__()` 方法中是可以被访问的，因为这个方法会在所有类主体执行前被执行。 但是类变量只能在元类的 `__new__()` 和 `__init__()` 方法中可见。

# `*args`和`**kwargs`的强制参数签名

In [65]:
from inspect import Signature, Parameter
parms = [ Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
    Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
    Parameter('z', Parameter.KEYWORD_ONLY, default=None) ]
sig = Signature(parms)
sig

<Signature (x, y=42, *, z=None)>

In [66]:
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
            for name in names]
    return Signature(parms)

class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

# Example use
class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')

class Point(Structure):
    __signature__ = make_sig('x', 'y')

In [68]:
import inspect
print(inspect.signature(Stock))
s1 = Stock('ACME', 100, 490.1)
s2 = Stock('ACME', 100)

(name, shares, price)


TypeError: missing a required argument: 'price'

In [69]:
s3 = Stock('ACME', 100, 490.1, shares=50)

TypeError: multiple values for argument 'shares'

In [70]:
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
            for name in names]
    return Signature(parms)

class StructureMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsdict['__signature__'] = make_sig(*clsdict.get('_fields',[]))
        return super().__new__(cls, clsname, bases, clsdict)

class Structure(metaclass=StructureMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

# Example
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
    _fields = ['x', 'y']

当我们自定义签名的时候，将签名存储在特定的属性 `__signature__` 中通常是很有用的。 这样的话，在使用 `inspect` 模块执行内省的代码就能发现签名并将它作为调用约定。

# 在类上强制使用编程规约

在元类中选择重新定义 `__new__()` 方法还是 `__init__()` 方法取决于你想怎样使用结果类。  
`__new__()` 方法在类创建之前被调用，通常用于通过某种方式（比如通过改变类字典的内容）修改类的定义。 而 `__init__()` 方法是在类被创建之后被调用，当你需要完整构建类对象的时候会很有用。  
在最后一个例子中，这是必要的，因为它使用了 `super()` 函数来搜索之前的定义。 它只能在类的实例被创建之后，并且相应的方法解析顺序也已经被设置好了。

## 拒绝任何有混合大小写名字作为方法的类定义

In [71]:
class NoMixedCaseMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        for name in clsdict:
            if name.lower() != name:
                raise TypeError('Bad attribute name: ' + name)
        return super().__new__(cls, clsname, bases, clsdict)

class Root(metaclass=NoMixedCaseMeta):
    pass

class A(Root):
    def foo_bar(self): # Ok
        pass

class B(Root):
    def fooBar(self): # TypeError
        pass

TypeError: Bad attribute name: fooBar

## 检测重载方法，确保它的调用参数跟父类中原始方法有着相同的参数签名。

In [72]:
from inspect import signature
import logging

class MatchSignaturesMeta(type):

    def __init__(self, clsname, bases, clsdict):
        super().__init__(clsname, bases, clsdict)
        sup = super(self, self)
        for name, value in clsdict.items():
            if name.startswith('_') or not callable(value):
                continue
            # Get the previous definition (if any) and compare the signatures
            prev_dfn = getattr(sup,name,None)
            if prev_dfn:
                prev_sig = signature(prev_dfn)
                val_sig = signature(value)
                if prev_sig != val_sig:
                    logging.warning('Signature mismatch in %s. %s != %s',
                                    value.__qualname__, prev_sig, val_sig)

# Example
class Root(metaclass=MatchSignaturesMeta):
    pass

class A(Root):
    def foo(self, x, y):
        pass

    def spam(self, x, *, z):
        pass

# Class with redefined methods, but slightly different signatures
class B(A):
    def foo(self, a, b):
        pass

    def spam(self,x,z):
        pass

# 以编程的方式定义类

`new_class()` 第四个参数最神秘，它是一个用来接受类命名空间的映射对象的函数。 通常这是一个普通的字典，但是它实际上是 `__prepare__()` 方法返回的任意对象

In [77]:
# stock.py
# Example of making a class manually from parts

# Methods
def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price
def cost(self):
    return self.shares * self.price

cls_dict = {
    '__init__' : __init__,
    'cost' : cost,
}

# Make a class
import types

Stock = types.new_class('Stock', (), {}, lambda ns: ns.update(cls_dict))
Stock.__module__ = __name__

In [78]:
s = Stock('acme',50,91.1)

每次当一个类被定义后，它的 `__module__` 属性包含定义它的模块名。 这个名字用于生成 `__repr__()` 方法的输出。它同样也被用于很多库，比如 `pickle` 。 因此，为了让你创建的类是“正确”的，你需要确保这个属性也设置正确了。

In [80]:
import abc
Stock = types.new_class('Stock', (), {'metaclass': abc.ABCMeta},lambda ns: ns.update(cls_dict))
Stock.__module__ = __name__
Stock,type(Stock)

(__main__.Stock, abc.ABCMeta)

In [88]:
class Base(metaclass=MyMeta,debug=True, synchronize=False):
    pass
class Spam(Base, debug=True, synchronize=False):
    pass
Spam = types.new_class('Spam', (Base,),{'debug': True, 'synchronize': False},lambda ns: ns.update(cls_dict))

通过调用 `sys._getframe()` 来获取调用者的模块名。

In [90]:
import operator
import types
import sys

def named_tuple(classname, fieldnames):
    # Populate a dictionary of field property accessors
    cls_dict = { name: property(operator.itemgetter(n))
                for n, name in enumerate(fieldnames) }

    # Make a __new__ function and add to the class dict
    def __new__(cls, *args):
        if len(args) != len(fieldnames):
            raise TypeError('Expected {} arguments'.format(len(fieldnames)))
        return tuple.__new__(cls, args)

    cls_dict['__new__'] = __new__

    # Make the class
    cls = types.new_class(classname, (tuple,), {},
                        lambda ns: ns.update(cls_dict))

    # Set the module to that of the caller
    cls.__module__ = sys._getframe(1).f_globals['__name__']
    return cls

In [92]:
import types
metaclass, kwargs, ns = types.prepare_class('Stock', (), {'metaclass': type})
metaclass, kwargs, ns

(type, {}, {})

```python
Stock = type('Stock', (), cls_dict)
```
这种方法的问题在于它忽略了一些关键步骤，比如对于元类中 `__prepare__()` 方法的调用。 
通过使用 `types.new_class()` ，你可以保证所有的必要初始化步骤都能得到执行。 比如，`types.new_class()` 第四个参数的回调函数接受 `__prepare__()` 方法返回的映射对象。

# 在定义的时候初始化类的成员

In [93]:
import operator

class StructTupleMeta(type):
    def __init__(cls, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for n, name in enumerate(cls._fields):
            setattr(cls, name, property(operator.itemgetter(n)))

class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields = []
    def __new__(cls, *args):
        if len(args) != len(cls._fields):
            raise ValueError('{} arguments required'.format(len(cls._fields)))
        return super().__new__(cls,args)

# 利用函数注解实现方法重载

## 简单方案

In [94]:
class Spam:
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)

    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)
s = Spam()
s.bar(1,2)
s.bar('a')

Bar 2: 1 2
Bar 2: a 0


元类和注解、描述器方案都不支持关键字参数和继承
+ 关键字参数位置比较混乱
+ 继承无法识别子类

## 元类和注解

In [95]:
# multiple.py
import inspect
import types

class MultiMethod:
    '''
    Represents a single multimethod.
    '''
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name

    def register(self, meth):
        '''
        Register a new method as a multimethod
        '''
        sig = inspect.signature(meth)

        # Build a type signature from the method's annotations
        types = []
        for name, parm in sig.parameters.items():
            if name == 'self':
                continue
            if parm.annotation is inspect.Parameter.empty:
                raise TypeError(
                    'Argument {} must be annotated with a type'.format(name)
                )
            if not isinstance(parm.annotation, type):
                raise TypeError(
                    'Argument {} annotation must be a type'.format(name)
                )
            if parm.default is not inspect.Parameter.empty:
                self._methods[tuple(types)] = meth
            types.append(parm.annotation)

        self._methods[tuple(types)] = meth

    def __call__(self, *args):
        '''
        Call a method based on type signature of the arguments
        '''
        types = tuple(type(arg) for arg in args[1:])
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            raise TypeError('No matching method for types {}'.format(types))

    def __get__(self, instance, cls):
        '''
        Descriptor method needed to make calls work in a class
        '''
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self

class MultiDict(dict):
    '''
    Special dictionary to build multimethods in a metaclass
    '''
    def __setitem__(self, key, value):
        if key in self:
            # If key already exists, it must be a multimethod or callable
            current_value = self[key]
            if isinstance(current_value, MultiMethod):
                current_value.register(value)
            else:
                mvalue = MultiMethod(key)
                mvalue.register(current_value)
                mvalue.register(value)
                super().__setitem__(key, mvalue)
        else:
            super().__setitem__(key, value)

class MultipleMeta(type):
    '''
    Metaclass that allows multiple dispatch of methods
    '''
    def __new__(cls, clsname, bases, clsdict):
        return type.__new__(cls, clsname, bases, dict(clsdict))

    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()

In [96]:
class Spam(metaclass=MultipleMeta):
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)

    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)

# Example: overloaded __init__
import time

class Date(metaclass=MultipleMeta):
    def __init__(self, year: int, month:int, day:int):
        self.year = year
        self.month = month
        self.day = day

    def __init__(self):
        t = time.localtime()
        self.__init__(t.tm_year, t.tm_mon, t.tm_mday)

## 描述器方案

In [101]:
import types

class multimethod:
    def __init__(self, func):
        self._methods = {}
        self.__name__ = func.__name__
        self._default = func

    def match(self, *types):
        def register(func):
            ndefaults = len(func.__defaults__) if func.__defaults__ else 0
            for n in range(ndefaults+1):
                self._methods[types[:len(types) - n]] = func
            return self
        return register

    def __call__(self, *args):
        types = tuple(type(arg) for arg in args[1:])
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            return self._default(*args)

    def __get__(self, instance, cls):
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self

In [102]:
class Spam:
    @multimethod
    def bar(self, *args):
        # Default method called if no match
        raise TypeError('No matching method for bar')

    @bar.match(int, int)
    def bar(self, x, y):
        print('Bar 1:', x, y)

    @bar.match(str, int)
    def bar(self, s, n = 0):
        print('Bar 2:', s, n)

# 避免重复的属性方法

In [103]:
class Person:
    def __init__(self, name ,age):
        self.name = name
        self.age = age

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('name must be a string')
        self._name = value

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, value):
        if not isinstance(value, int):
            raise TypeError('age must be an int')
        self._age = value

In [104]:
def typed_property(name, expected_type):
    storage_name = '_' + name

    @property
    def prop(self):
        return getattr(self, storage_name)

    @prop.setter
    def prop(self, value):
        if not isinstance(value, expected_type):
            raise TypeError('{} must be a {}'.format(name, expected_type))
        setattr(self, storage_name, value)

    return prop

# Example use
class Person:
    name = typed_property('name', str)
    age = typed_property('age', int)

    def __init__(self, name, age):
        self.name = name
        self.age = age

In [105]:
from functools import partial

String = partial(typed_property, expected_type=str)
Integer = partial(typed_property, expected_type=int)

# Example:
class Person:
    name = String('name')
    age = Integer('age')

    def __init__(self, name, age):
        self.name = name
        self.age = age

# 定义上下文管理器的简单方法

使用 `contexlib` 模块中的 `@contextmanager` 装饰器

## 计时器

In [108]:
import time
from contextlib import contextmanager

@contextmanager
def timethis(label):
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
        print('{}: {}'.format(label, end - start))

# Example use
with timethis('counting'):
    n = 10000000
    while n > 0:
        n -= 1

counting: 1.528010368347168


## 列表事务

In [109]:
@contextmanager
def list_transaction(orig_list):
    working = list(orig_list)
    yield working
    orig_list[:] = working

In [111]:
items = [1, 2, 3]
with list_transaction(items) as working:
    working.append(4)
    working.append(5)
items

[1, 2, 3, 4, 5]

In [114]:
with list_transaction(items) as working:
    working.append(6)
    working.append(7)
    raise RuntimeError('oops')

RuntimeError: oops

In [115]:
items

[1, 2, 3, 4, 5]

In [110]:
import time

class timethis:
    def __init__(self, label):
        self.label = label

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, exc_ty, exc_val, exc_tb):
        end = time.time()
        print('{}: {}'.format(self.label, end - self.start))

# 在局部变量域中执行代码

In [117]:
def test():
    a = 13
    exec('b = a + 1')
    print(b)

In [118]:
test()

NameError: name 'b' is not defined

In [119]:
def test():
    a  = 13
    loc = locals()
    exec('b = a  + 1')
    b = loc['b']
    print(b)

In [120]:
test()

14


在函数里面， 传递给 `exec()` 的局部范围是拷贝实际局部变量组成的一个字典。 因此，如果 `exec()` 如果执行了修改操作，这种修改后的结果对实际局部变量值是没有影响的。  
调用 `locals()` 获取局部变量时，获得的是传递给 `exec()` 的局部变量的一个拷贝。通过在代码执行后审查这个字典的值，那就能获取修改后的值了。  
`locals()` 会获取局部变量值中的值并覆盖字典中相应的变量。  
作为 `locals()` 的一个替代方案，你可以使用你自己的字典，并将它传递给 `exec()`

In [121]:
def test1():
    x = 0
    exec('x += 1')
    print(x)

In [122]:
test1()

0


In [124]:
def test3():
    x = 0 
    loc = locals()
    print(loc)
    exec('x+=1')
    print(loc)
    locals()
    print(loc)

In [126]:
test3()

{'x': 0}
{'x': 1, 'loc': {...}}
{'x': 0, 'loc': {...}}


In [127]:
def test4():
    a = 13 
    loc = {'a':a}
    glb = {}
    exec('b=a+1',glb,loc)
    b = loc['b']
    print(b)

In [128]:
test4()

14


# 解析与分析Python源码

In [130]:
x = 42
print(eval('2 + 3 *4 +x'))
exec('for i in range(10):print(i)')

56
0
1
2
3
4
5
6
7
8
9


## `ast`解析

In [131]:
import ast 
ex = ast.parse('2 + 3*4 + x', mode='eval')
ex

In [132]:
ast.dump(ex)

"Expression(body=BinOp(left=BinOp(left=Constant(value=2, kind=None), op=Add(), right=BinOp(left=Constant(value=3, kind=None), op=Mult(), right=Constant(value=4, kind=None))), op=Add(), right=Name(id='x', ctx=Load())))"

In [133]:
top = ast.parse('for i in range(10): print(i)', mode='exec')
top

In [134]:
ast.dump(top)

"Module(body=[For(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[Constant(value=10, kind=None)], keywords=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='i', ctx=Load())], keywords=[]))], orelse=[], type_comment=None)], type_ignores=[])"

In [135]:
import ast

class CodeAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.loaded = set()
        self.stored = set()
        self.deleted = set()

    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Load):
            self.loaded.add(node.id)
        elif isinstance(node.ctx, ast.Store):
            self.stored.add(node.id)
        elif isinstance(node.ctx, ast.Del):
            self.deleted.add(node.id)

In [136]:
# Some Python code
code = '''
for i in range(10):
    print(i)
del i
'''

# Parse into an AST
top = ast.parse(code, mode='exec')

# Feed the AST to analyze name usage
c = CodeAnalyzer()
c.visit(top)
print('Loaded:', c.loaded)
print('Stored:', c.stored)
print('Deleted:', c.deleted)

Loaded: {'print', 'i', 'range'}
Stored: {'i'}
Deleted: {'i'}


In [138]:
exec(compile(top,'<stdin>', 'exec'))

0
1
2
3
4
5
6
7
8
9


## 装饰器:通过重新解析函数体源码、 重写AST并重新创建函数代码对象来将全局访问变量降为函数体作用范围

In [146]:
# namelower.py
import ast
import inspect

# Node visitor that lowers globally accessed names into
# the function body as local variables.
class NameLower(ast.NodeVisitor):
    def __init__(self, lowered_names):
        self.lowered_names = lowered_names

    def visit_FunctionDef(self, node):
        # Compile some assignments to lower the constants
        code = '__globals = globals()\n'
        code += '\n'.join("{0} = __globals['{0}']".format(name)
                            for name in self.lowered_names)
        code_ast = ast.parse(code, mode='exec')

        # Inject new statements into the function body
        node.body[:0] = code_ast.body

        # Save the function object
        self.func = node

# Decorator that turns global names into locals
def lower_names(*namelist):
    def lower(func):
        srclines = inspect.getsource(func).splitlines()
        # Skip source lines prior to the @lower_names decorator
        for n, line in enumerate(srclines):
            if '@lower_names' in line:
                break

        src = '\n'.join(srclines[n+1:])
        # Hack to deal with indented code
        if src.startswith((' ','\t')):
            src = 'if 1:\n' + src
        top = ast.parse(src, mode='exec')

        # Transform the AST
        cl = NameLower(namelist)
        cl.visit(top)

        # Execute the modified AST
        temp = {}
        exec(compile(top,'','exec'), temp, temp)

        # Pull out the modified code object
        func.__code__ = temp[func.__name__].__code__
        return func
    return lower

In [152]:
INCR = 1
@lower_names('INCR')
def countdown(n):
    while n > 0:
        n -= INCR

In [153]:
def countdown(n):
    __globals = globals()
    INCR = __globals['INCR']
    while n > 0:
        n -= INCR

In [155]:
def countdown1(n):
    while n > 0:
        n -= INCR

In [157]:
from timeit import timeit
timeit('countdown(1000)','from __main__ import countdown')

47.48684930600575

In [158]:
from timeit import timeit
timeit('countdown1(1000)','from __main__ import countdown1')

50.49969709399738

# 拆解Python字节码

## `dis` 模块可以被用来输出任何Python函数的反编译结果

In [160]:
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
    print('Blastoff!')

In [169]:
import dis
dis.dis(countdown)

  2     >>    0 LOAD_FAST                0 (n)
              2 LOAD_CONST               1 (0)
              4 COMPARE_OP               4 (>)
              6 POP_JUMP_IF_FALSE       28

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_CONST               2 ('T-minus')
             12 LOAD_FAST                0 (n)
             14 CALL_FUNCTION            2
             16 POP_TOP

  4          18 LOAD_FAST                0 (n)
             20 LOAD_CONST               3 (1)
             22 INPLACE_SUBTRACT
             24 STORE_FAST               0 (n)
             26 JUMP_ABSOLUTE            0

  5     >>   28 LOAD_GLOBAL              0 (print)
             30 LOAD_CONST               4 ('Blastoff!')
             32 CALL_FUNCTION            1
             34 POP_TOP
             36 LOAD_CONST               0 (None)
             38 RETURN_VALUE


In [163]:
# dix()函数解析的原始字节码
countdown.__code__.co_code

b'|\x00d\x01k\x04r\x1ct\x00d\x02|\x00\x83\x02\x01\x00|\x00d\x038\x00}\x00q\x00t\x00d\x04\x83\x01\x01\x00d\x00S\x00'

In [164]:
c = countdown.__code__.co_code

In [168]:
import opcode
opcode.opname[c[0]],opcode.opname[c[1]],opcode.opname[c[2]]

('LOAD_FAST', '<0>', 'LOAD_CONST')

## 将原始字节码序列转换成`opcodes`和参数

In [171]:
import opcode

def generate_opcodes(codebytes):
    extended_arg = 0
    i = 0
    n = len(codebytes)
    while i < n:
        op = codebytes[i]
        i += 1
        if op >= opcode.HAVE_ARGUMENT:
            oparg = codebytes[i] + codebytes[i+1]*256 + extended_arg
            extended_arg = 0
            i += 2
            if op == opcode.EXTENDED_ARG:
                extended_arg = oparg * 65536
                continue
        else:
            oparg = None
        yield (op, oparg)

In [178]:
for op, oparg in generate_opcodes(countdown.__code__.co_code):
    print(op, opcode.opname[op], oparg)

124 LOAD_FAST 25600
1 POP_TOP None
107 COMPARE_OP 29188
28 INPLACE_FLOOR_DIVIDE None
116 LOAD_GLOBAL 25600
2 ROT_TWO None
124 LOAD_FAST 33536
2 ROT_TWO None
1 POP_TOP None
0 <0> None
124 LOAD_FAST 25600
3 ROT_THREE None
56 INPLACE_SUBTRACT None
0 <0> None
125 STORE_FAST 28928
0 <0> None
116 LOAD_GLOBAL 25600
4 DUP_TOP None
131 CALL_FUNCTION 257
0 <0> None
100 LOAD_CONST 21248
0 <0> None
